Setting up the project



In [1]:
!pip install -q openai google-generativeai transformers torch

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "your-key"  # Replace with your OpenAI key

Adding Sample reviews for Sentiment Analysis

In [3]:
reviews = [
    "This movie was absolutely fantastic! I loved every minute.",
    "The service was terrible, waited an hour for cold food.",
    "Neutral product – does the job but nothing special.",
    "Best purchase ever, highly recommend to everyone!",
    "Disappointing quality, broke after one use."
]

OpenAI Basic Sentiment Analysis

In [4]:
from openai import OpenAI
client = OpenAI()

def openai_sentiment(text):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": f"Analyze the sentiment of this text (positive, negative, or neutral) and give a brief explanation: {text}"}],
        temperature=0.5
    )
    return response.choices[0].message.content.strip()

In [5]:
openai_results = [openai_sentiment(review) for review in reviews]
print("OpenAI Results:")
for review, result in zip(reviews, openai_results):
    print(f"Review: {review}\nSentiment: {result}\n---")

OpenAI Results:
Review: This movie was absolutely fantastic! I loved every minute.
Sentiment: The sentiment of the text is **positive**. The use of strong adjectives like "absolutely fantastic" and the phrase "I loved every minute" indicate a high level of enjoyment and satisfaction with the movie. The overall tone conveys enthusiasm and appreciation.
---
Review: The service was terrible, waited an hour for cold food.
Sentiment: The sentiment of the text is negative. 

Explanation: The speaker expresses dissatisfaction with the service they received, specifically highlighting a long wait time and the quality of the food as being poor (cold). The use of the word "terrible" directly conveys a negative experience, indicating frustration and disappointment.
---
Review: Neutral product – does the job but nothing special.
Sentiment: The sentiment of the text is neutral. 

Explanation: The phrase "does the job" indicates that the product fulfills its intended purpose, suggesting a basic level

Hugging Face Transformers – Load and Run Sentiment Model

In [6]:
from transformers import pipeline

# Load pre-trained sentiment model (free, runs locally in Colab)
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [7]:
hf_results = sentiment_pipeline(reviews)
print("Hugging Face Results:")
for review, result in zip(reviews, hf_results):
    print(f"Review: {review}\nSentiment: {result['label']} (Score: {result['score']:.4f})\n---")

Hugging Face Results:
Review: This movie was absolutely fantastic! I loved every minute.
Sentiment: POSITIVE (Score: 0.9999)
---
Review: The service was terrible, waited an hour for cold food.
Sentiment: NEGATIVE (Score: 0.9996)
---
Review: Neutral product – does the job but nothing special.
Sentiment: NEGATIVE (Score: 0.9962)
---
Review: Best purchase ever, highly recommend to everyone!
Sentiment: POSITIVE (Score: 0.9999)
---
Review: Disappointing quality, broke after one use.
Sentiment: NEGATIVE (Score: 0.9997)
---


In [13]:
import pandas as pd

data = {
    "Review": reviews,
    "OpenAI (GPT-4o-mini)": openai_results,
    "Hugging Face (DistilBERT)": [f"{r['label']} (Score: {r['score']:.4f})" for r in hf_results]
}
df = pd.DataFrame(data)
print("Sentiment Comparison Table")
print(df.to_markdown(index=False))

Sentiment Comparison Table
| Review                                                     | OpenAI (GPT-4o-mini)                                                                                                                                                                                                                                                                                                                                                                                                                   | Hugging Face (DistilBERT)   |
|:-----------------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------